# 01. Buffer Memory (전체 대화 저장)

이 노트북에서는 **Buffer Memory** 방식을 LangChain과 LangGraph로 구현합니다.

**Buffer Memory란?**
- 모든 대화 내용을 그대로 저장
- 가장 단순하지만, 대화가 길어지면 토큰 비용 증가

**학습 목표:**
- LangChain ChatMessageHistory 사용법
- LangGraph 그래프 구성
- LangGraph MemorySaver 활용

---

## 1. 패키지 설치

In [29]:
!pip install langchain langchain-anthropic langgraph langgraph-checkpoint-sqlite -q
print("설치 완료!")

설치 완료!


---

## 2. 모델 선택

Claude 4.5 모델 중 하나를 선택하세요.

| 모델 | 특징 | 비용 |
|------|------|------|
| Haiku 4.5 | 가장 빠름, 간단한 작업 | 저렴 |
| Sonnet 4.5 | 균형 잡힌 성능 | 중간 |
| Opus 4.5 | 최고 성능 | 높음 |

In [27]:
# ===== Claude 4.5 모델 선택 =====
# 아래 세 가지 모델 중 하나를 선택하세요 (주석 해제)

# 1. Haiku 4.5 - 가장 빠르고 저렴 (테스트용 추천)
MODEL = "claude-haiku-4-5-20251001"

# 2. Sonnet 4.5 - 균형 잡힌 성능
# MODEL = "claude-sonnet-4-5-20250929"

# 3. Opus 4.5 - 최고 성능 (비용 높음)
# MODEL = "claude-opus-4-5-20251101"

print(f"선택된 모델: {MODEL}")

선택된 모델: claude-haiku-4-5-20251001


---

## 3. API 키 설정

In [28]:
import os

# 방법 1: 직접 입력
# os.environ["ANTHROPIC_API_KEY"] = "sk-ant-여기에-API-키-입력"

# 방법 2: Colab Secrets
try:
    from google.colab import userdata
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
    print("Colab Secrets에서 API 키를 불러왔습니다!")
except:
    print("직접 API 키를 입력하세요.")

Colab Secrets에서 API 키를 불러왔습니다!


---

## 4. LangChain으로 Buffer Memory 구현

### 4.1 기본 구성요소

In [31]:
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.chat_history import InMemoryChatMessageHistory

# LLM 초기화
llm = ChatAnthropic(
    model=MODEL,
    max_tokens=500
)

# Buffer Memory: 모든 대화를 저장
chat_history = InMemoryChatMessageHistory()

print("LangChain 구성요소 초기화 완료!")

LangChain 구성요소 초기화 완료!


### 4.2 Buffer Memory 챗봇 함수

In [32]:
def chat_with_buffer_memory(user_input: str) -> str:
    """
    Buffer Memory 방식 챗봇.
    모든 대화 내용을 그대로 저장하고 전달합니다.
    """
    # 사용자 메시지 추가
    chat_history.add_user_message(user_input)

    # 시스템 메시지 + 전체 대화 히스토리
    messages = [
        SystemMessage(content="당신은 친절한 AI 어시스턴트입니다. 한국어로 답변하세요."),
        *chat_history.messages  # 모든 대화 포함
    ]

    # LLM 호출
    response = llm.invoke(messages)

    # AI 응답 저장
    chat_history.add_ai_message(response.content)

    return response.content


# 테스트
print("="*50)
print("Buffer Memory 테스트")
print("="*50)

# 대화 1
response1 = chat_with_buffer_memory("안녕! 내 이름은 철수야.")
print(f"\n[1] 사용자: 안녕! 내 이름은 철수야.")
print(f"    AI: {response1}")

# 대화 2
response2 = chat_with_buffer_memory("오늘 날씨가 좋네.")
print(f"\n[2] 사용자: 오늘 날씨가 좋네.")
print(f"    AI: {response2}")

# 대화 3 - 이전 대화 기억 확인
response3 = chat_with_buffer_memory("내 이름이 뭐라고 했지?")
print(f"\n[3] 사용자: 내 이름이 뭐라고 했지?")
print(f"    AI: {response3}")

Buffer Memory 테스트

[1] 사용자: 안녕! 내 이름은 철수야.
    AI: 안녕하세요, 철수님! 만나서 반갑습니다. 😊

저는 당신을 도와드릴 준비가 되어 있는 AI 어시스턴트입니다. 궁금한 점이나 도움이 필요하신 것이 있으시면 편하게 말씀해 주세요!

오늘 무엇을 도와드릴까요?

[2] 사용자: 오늘 날씨가 좋네.
    AI: 네, 정말 좋은 날씨네요! 😊

다만 저는 실시간 날씨 정보에 접근할 수 없어서, 철수님이 계신 지역의 정확한 날씨를 확인해 드릴 수는 없습니다.

하지만 좋은 날씨라니 정말 좋으시겠네요! 이런 날씨에는:
- 산책이나 운동하기 좋은 시간
- 야외 활동을 즐기기 좋은 날
- 기분도 좋아지는 날씨

이런 좋은 날씨를 잘 즐기시길 바랍니다! 혹시 특별히 계획하고 계신 활동이 있으신가요? 😄

[3] 사용자: 내 이름이 뭐라고 했지?
    AI: 당신의 이름은 **철수**라고 하셨어요! 😊

처음 인사할 때 "내 이름은 철수야"라고 말씀해 주셨습니다.

혹시 제가 잘못 기억하고 있는 건가요?


### 4.3 메모리 상태 확인

In [41]:
print("현재 저장된 대화 수:", len(chat_history.messages))
print("\n저장된 대화 내용:")
print("-" * 50)

for i, msg in enumerate(chat_history.messages, 1):
    role = "사용자" if isinstance(msg, HumanMessage) else "AI"
    print(f"[{i}] {role}: {msg.content[:50]}..." if len(msg.content) > 50 else f"[{i}] {role}: {msg.content}")

현재 저장된 대화 수: 6

저장된 대화 내용:
--------------------------------------------------
[1] 사용자: 안녕! 내 이름은 철수야.
[2] AI: 안녕하세요, 철수님! 만나서 반갑습니다. 😊

저는 당신을 도와드릴 준비가 되어 있는 AI...
[3] 사용자: 오늘 날씨가 좋네.
[4] AI: 네, 정말 좋은 날씨네요! 😊

다만 저는 실시간 날씨 정보에 접근할 수 없어서, 철수님이...
[5] 사용자: 내 이름이 뭐라고 했지?
[6] AI: 당신의 이름은 **철수**라고 하셨어요! 😊

처음 인사할 때 "내 이름은 철수야"라고 말...


---

## 5. LangGraph로 Buffer Memory 구현

LangGraph는 상태 관리와 메모리 저장을 더 체계적으로 할 수 있습니다.

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

# 상태 정의: 메시지 리스트를 관리
class State(TypedDict):
    messages: Annotated[list, add_messages]  # 메시지 자동 누적

# LLM 노드
def chatbot_node(state: State) -> dict:
    """LLM을 호출하여 응답 생성"""
    messages = [
        SystemMessage(content="당신은 친절한 AI 어시스턴트입니다. 한국어로 답변하세요."),
        *state["messages"]
    ]
    response = llm.invoke(messages)
    return {"messages": [response]}

# 그래프 구성
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# MemorySaver: 대화 상태를 메모리에 저장
memory_saver = MemorySaver()

# 그래프 컴파일 (메모리 저장 활성화)
graph = graph_builder.compile(checkpointer=memory_saver)

print("LangGraph 그래프 구성 완료!")

LangGraph 그래프 구성 완료!


### 5.1 LangGraph 챗봇 테스트

In [35]:
# 세션 ID로 대화 구분
config = {"configurable": {"thread_id": "user-session-1"}}

def chat_with_langgraph(user_input: str) -> str:
    """LangGraph 기반 Buffer Memory 챗봇"""
    result = graph.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config=config
    )
    return result["messages"][-1].content


print("="*50)
print("LangGraph Buffer Memory 테스트")
print("="*50)

# 대화 1
response1 = chat_with_langgraph("안녕! 나는 영희야.")
print(f"\n[1] 사용자: 안녕! 나는 영희야.")
print(f"    AI: {response1}")

# 대화 2
response2 = chat_with_langgraph("오늘 뭐 하면 좋을까?")
print(f"\n[2] 사용자: 오늘 뭐 하면 좋을까?")
print(f"    AI: {response2}")

# 대화 3 - 이전 대화 기억 확인
response3 = chat_with_langgraph("아까 내가 이름을 뭐라고 했지?")
print(f"\n[3] 사용자: 아까 내가 이름을 뭐라고 했지?")
print(f"    AI: {response3}")

LangGraph Buffer Memory 테스트

[1] 사용자: 안녕! 나는 영희야.
    AI: 안녕하세요, 영희님! 😊

같은 패턴으로 반복되고 있네요. 혹시 제가 대화를 제대로 기억하고 있는지 확인하고 싶으신 건가요?

저는 이전 대화 내용을 기억하고 있습니다:
- 영희님이 처음 소개해주셨어요
- 오늘 뭘 하면 좋을까 물어봐주셨어요
- 제 이름이 뭐였는지 확인해주셨어요

혹시 다른 주제로 대화하고 싶으신가요? 아니면 제 기억력을 테스트해주시는 걸까요? 😄

편하게 말씀해주세요!

[2] 사용자: 오늘 뭐 하면 좋을까?
    AI: 영희님, 같은 질문을 반복해주시고 계시네요! 😊

혹시 제가 놓친 게 있나요? 아니면:

1. **더 구체적인 정보를 원하시나요?**
   - 현재 기분이 어떠신가요?
   - 날씨는 어떤가요?
   - 오늘 얼마나 시간이 있나요?

2. **특정 분야에 집중하고 싶으신가요?**
   - 운동, 문화생활, 공부, 휴식 등

3. **혹시 저를 테스트 중이신가요?** 😄

구체적으로 말씀해주시면 더 맞춤형 제안을 드릴 수 있어요! 무엇을 도와드릴까요?

[3] 사용자: 아까 내가 이름을 뭐라고 했지?
    AI: 영희님, 당신이 말씀해주신 이름은 **영희**예요! 😊

그런데 지금 같은 질문들을 반복해주고 계시는데, 혹시:

1. **제가 제대로 기억하고 있는지 확인**하고 싶으신 건가요?
2. **대화의 패턴을 테스트**하고 싶으신 건가요?
3. **뭔가 다른 걸 원하신데** 어떻게 표현해야 할지 고민 중이신가요?

편하게 말씀해주시면 좋겠습니다! 😊 
혹은 완전히 새로운 주제로 대화하고 싶으신 거라면, 언제든지 말씀해주세요!


### 5.2 저장된 상태 확인

In [37]:
# 현재 저장된 상태 조회
state_snapshot = graph.get_state(config)

print("현재 저장된 메시지 수:", len(state_snapshot.values["messages"]))
print("\n저장된 대화:")
print("-" * 50)

for i, msg in enumerate(state_snapshot.values["messages"], 1):
    role = "사용자" if isinstance(msg, HumanMessage) else "AI"
    content = msg.content[:60] + "..." if len(msg.content) > 60 else msg.content
    print(f"[{i}] {role}: {content}")

현재 저장된 메시지 수: 18

저장된 대화:
--------------------------------------------------
[1] 사용자: 안녕! 나는 영희야.
[2] AI: 안녕하세요, 영희님! 만나서 반갑습니다. 😊

무엇을 도와드릴까요?
[3] 사용자: 오늘 뭐 하면 좋을까?
[4] AI: 오늘 어떤 활동을 하면 좋을지는 여러 요소에 따라 달라질 것 같아요! 몇 가지 제안을 드릴게요:

**날씨가...
[5] 사용자: 아까 내가 이름을 뭐라고 했지?
[6] AI: 아까 당신이 말씀해주신 이름은 **영희**예요! 😊

처음 인사할 때 "나는 영희야"라고 소개해주셨죠.
[7] 사용자: 안녕! 나는 영희야.
[8] AI: 안녕하세요, 영희님! 😊

아, 혹시 다시 인사해주시는 건가요? 아니면 뭔가 새로운 것을 시작하고 싶으신 건...
[9] 사용자: 오늘 뭐 하면 좋을까?
[10] AI: 오늘 어떤 활동을 하면 좋을지는 여러 요소에 따라 달라질 것 같아요! 몇 가지 제안을 드릴게요:

**날씨가...
[11] 사용자: 아까 내가 이름을 뭐라고 했지?
[12] AI: 아까 당신이 말씀해주신 이름은 **영희**예요! 😊

처음 인사할 때 "나는 영희야"라고 소개해주셨죠.
[13] 사용자: 안녕! 나는 영희야.
[14] AI: 안녕하세요, 영희님! 😊

같은 패턴으로 반복되고 있네요. 혹시 제가 대화를 제대로 기억하고 있는지 확인하고...
[15] 사용자: 오늘 뭐 하면 좋을까?
[16] AI: 영희님, 같은 질문을 반복해주시고 계시네요! 😊

혹시 제가 놓친 게 있나요? 아니면:

1. **더 구체적...
[17] 사용자: 아까 내가 이름을 뭐라고 했지?
[18] AI: 영희님, 당신이 말씀해주신 이름은 **영희**예요! 😊

그런데 지금 같은 질문들을 반복해주고 계시는데, 혹...


---

## 6. SQLite 기반 영구 저장

LangGraph의 `SqliteSaver`를 사용하면 대화를 파일로 저장할 수 있습니다.

In [38]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

# SQLite 연결 생성
conn = sqlite3.connect("chat_memory.db", check_same_thread=False)
sqlite_saver = SqliteSaver(conn)

# 새 그래프 컴파일 (SQLite 저장)
graph_with_sqlite = graph_builder.compile(checkpointer=sqlite_saver)

print("SQLite 기반 영구 저장 설정 완료!")
print("파일: chat_memory.db")

SQLite 기반 영구 저장 설정 완료!
파일: chat_memory.db


In [39]:
# SQLite 저장 테스트
config_sqlite = {"configurable": {"thread_id": "persistent-session-1"}}

# 대화 실행
result = graph_with_sqlite.invoke(
    {"messages": [HumanMessage(content="안녕! 이 대화는 파일에 저장돼.")]},
    config=config_sqlite
)

print("응답:", result["messages"][-1].content)
print("\n대화가 chat_memory.db에 저장되었습니다.")

응답: 안녕하세요! 👋

네, 이 대화가 파일에 저장된다는 점 알겠습니다.

혹시 이번에는 구체적으로 어떤 도움을 드릴 수 있을까요? 궁금한 점이나 필요한 것이 있으시면 편하게 말씀해 주세요! 😊

대화가 chat_memory.db에 저장되었습니다.


---

## 7. Buffer Memory 특징

### 장점
- 구현이 간단함
- 모든 맥락 유지
- 디버깅 용이

### 단점
- 대화가 길어지면 토큰 비용 급증
- 모델의 컨텍스트 윈도우 초과 가능

### 토큰 누적 예시

| 대화 턴 | 대략적인 입력 토큰 |
|--------|------------------|
| 1번째 | ~100 |
| 5번째 | ~500 |
| 10번째 | ~1,000+ |
| 20번째 | ~2,000+ |

### 다음 단계

**02_window_memory.ipynb**에서는 최근 N개의 대화만 저장하는 Window Memory를 배웁니다.

In [40]:
# 정리: SQLite 연결 닫기
conn.close()
print("정리 완료!")

정리 완료!
